This program is to try to solve GrossPitaevskii equation by using Crank-Nicholson in 2D

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import scipy.linalg as la

class CrankNicolson2D: